In [1]:
!pip install -qU langchain openai tiktoken cohere \
                 "pinecone-client[grpc]" pinecone-text \
                  python-dotenv bs4 nest_asyncio \


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Set project directory.
import os, sys
proj_dir = '/content/drive/MyDrive/LLM/pinecone-hack'
os.chdir(os.path.join(proj_dir))
os.path.dirname(os.path.realpath(__name__))

'/content/drive/MyDrive/LLM/pinecone-hack'

In [3]:
import os
from dotenv import load_dotenv

# loads environment variables
load_dotenv()

# initialize connection to pinecone (get API key at app.pinecone.io)
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY") or "PINECONE_API_KEY"
# find your environment next to the api key in pinecone console
PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT") or "PINECONE_ENVIRONMENT"

COHERE_API_KEY = os.getenv("COHERE_API_KEY") or "COHERE_API_KEY"

## Data Preparation

In [29]:
import pandas as pd

data_path = "./data/listings_1.csv"

columns = ['id', 'name', 'description', 'neighborhood_overview',
       'picture_url', 'host_name', 'host_since', 'host_about',
       'host_is_superhost', 'neighbourhood', 'property_type', 'room_type',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'amenities',
       'price', 'minimum_nights', 'maximum_nights',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value']

df = pd.read_csv(data_path, usecols=columns, dtype=str, na_values="")

df = df.fillna("")

df.head()


,id,name,description,neighborhood_overview,picture_url,host_name,host_since,host_about,host_is_superhost,neighbourhood,...,price,minimum_nights,maximum_nights,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value
0,30216638,Ama Hostel Bangkok - Private Room For 8 Person,Our mixed dormitory bed is comfortable and cle...,"Ama Hostel Bangkok is set in Bangkok, 700 m fr...",https://a0.muscache.com/pictures/2856e97a-827e...,Nakun,2018-11-22,,f,Samphanthawong,...,"$4,800.00",1,365,,,,,,,
1,30387182,Ama Hostel Bangkok - Quadruple Room Shared Bath,"Experience Culture, Street Food, Heritage Thai...","Ama Hostel Bangkok is set in Bangkok, 700 m fr...",https://a0.muscache.com/pictures/9777182f-7b09...,Nakun,2018-11-22,,f,Samphanthawong,...,"$2,800.00",1,365,5,5,5,5,4,4,5
2,35190417,A Bit of Heaven in Bangkok,Luxurious Condo is in Supalai Wellington on th...,A wide variety of art and cultural shows and p...,https://a0.muscache.com/pictures/efde2ee7-bbb6...,Wilts,2016-10-17,,f,Huai Khwang,...,"$2,570.00",30,365,,,,,,,
3,31450489,Warm two-bedroom,"The senior designer jointly designed, simple a...",,https://a0.muscache.com/pictures/c3cb190d-84a1...,ยานนาวา,2019-01-10,,f,Khan Na Yao,...,"$2,570.00",1,30,5,5,5,5,5,5,5
4,31196662,Supalai Wellington 2,"Convenience location, Near many Department sto...","New Central Rama 9 Department stores, Fortunes...",https://a0.muscache.com/pictures/b78e6421-1bcf...,Sumittra,2015-02-14,,f,Huai Khwang,...,$890.00,30,120,,,,,,,


In [31]:
df.amenities[:5]

0    ["Hair dryer", "Essentials", "Wifi", "Air cond...
1    ["TV", "Essentials", "Wifi", "Air conditioning...
2    ["TV", "Essentials", "Kitchen", "Hot tub", "Wi...
3    ["TV", "Essentials", "Kitchen", "Carbon monoxi...
4    ["TV", "Essentials", "Kitchen", "Gym", "Privat...
Name: amenities, dtype: object

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17396 entries, 0 to 17395
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   id                           17396 non-null  object
 1   name                         17388 non-null  object
 2   description                  16826 non-null  object
 3   neighborhood_overview        8847 non-null   object
 4   picture_url                  17382 non-null  object
 5   host_name                    17392 non-null  object
 6   host_since                   17395 non-null  object
 7   host_about                   10337 non-null  object
 8   host_is_superhost            17383 non-null  object
 9   neighbourhood                17347 non-null  object
 10  property_type                17369 non-null  object
 11  room_type                    17386 non-null  object
 12  accommodates                 17392 non-null  object
 13  bathrooms                    17

In [34]:
from bs4 import BeautifulSoup

def html_to_text(html):

  return BeautifulSoup(html).get_text()

In [38]:
df.description = df.description.apply(html_to_text)
df.neighborhood_overview = df.neighborhood_overview.apply(html_to_text)

df.description[12]

<ipython-input-34-cc18447f7f5e>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(html).get_text()
<ipython-input-34-cc18447f7f5e>:5: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  return BeautifulSoup(html).get_text()


'2 Queen bed for 2 persons.Shared bathroom is right in front of the room. Very clean and convenient.The property is a 5-minute walk to Rama 9 MRT Subway Station, Fortune Town and Central Department Store. It is 30-minute drive to Suvarnabhumi or Donmueng Airport.The spaceOur core value is to be a warmly welcomed home and to deliver great experience during your stay in Bangkok. No matter how your trip is like, once you step in our hostel, you will feel like arriving home.  We also guarantee for the clean room, comfortable bed, good location, and helpful staff.Guest access- Common area- Common kitchen- Free board game'

In [41]:
df.host_is_superhost = df.host_is_superhost.apply(lambda x: "Yes" if x=="t" else "No")
df.host_is_superhost[100]

'Yes'

In [45]:
df.amenities = df.amenities.apply(lambda x: x.replace('[', '').replace(']', ''))
df.amenities[:5]

0    "Hair dryer", "Essentials", "Wifi", "Air condi...
1    "TV", "Essentials", "Wifi", "Air conditioning"...
2    "TV", "Essentials", "Kitchen", "Hot tub", "Wif...
3    "TV", "Essentials", "Kitchen", "Carbon monoxid...
4    "TV", "Essentials", "Kitchen", "Gym", "Private...
Name: amenities, dtype: object

In [46]:
df.amenities = df.amenities.apply(lambda x: x.replace('"', ''))
df.amenities[:5]

0    Hair dryer, Essentials, Wifi, Air conditioning...
1    TV, Essentials, Wifi, Air conditioning, Lock o...
2    TV, Essentials, Kitchen, Hot tub, Wifi, Gym, A...
3    TV, Essentials, Kitchen, Carbon monoxide alarm...
4    TV, Essentials, Kitchen, Gym, Private living r...
Name: amenities, dtype: object

In [47]:
df.to_csv('./data/listings_1_clean.csv', index=False)

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17396 entries, 0 to 17395
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   id                           17396 non-null  object
 1   name                         17396 non-null  object
 2   description                  17396 non-null  object
 3   neighborhood_overview        17396 non-null  object
 4   picture_url                  17396 non-null  object
 5   host_name                    17396 non-null  object
 6   host_since                   17396 non-null  object
 7   host_about                   17396 non-null  object
 8   host_is_superhost            17396 non-null  object
 9   neighbourhood                17396 non-null  object
 10  property_type                17396 non-null  object
 11  room_type                    17396 non-null  object
 12  accommodates                 17396 non-null  object
 13  bathrooms                    17

## Cohere Embeddings

In [4]:
from langchain.embeddings import CohereEmbeddings

# model = "embed-english-v2.0", default (dim 4096)
embeddings = CohereEmbeddings(model = 'small',
                              cohere_api_key=os.environ['COHERE_API_KEY'])

text = "This is a test document."

doc_result = embeddings.embed_documents([text])

print(len(doc_result), ',', len(doc_result[0]))

# query_result = embeddings.embed_query(text)

1 , 1024


## Connect to Pinecone

In [5]:
import pinecone

# init connection to pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRONMENT
)

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [6]:
# choose a name for your index
index_name = "airbnb-retrieval"

if index_name not in pinecone.list_indexes():
    # create the index
    pinecone.create_index(
      index_name,
      dimension=1024
    )

index = pinecone.GRPCIndex(index_name)
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [49]:
len(df)

17396

In [ ]:
df.to_dict()

In [51]:
from tqdm.auto import tqdm

batch_size = 100

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(df))
    # extract metadata batch
    meta_batch = df.iloc[i:i_end]
    meta_dict = meta_batch.to_dict(orient="records")
    # concatinate all metadata field except for id and year to form a single string
    meta_batch = [" ".join(x) for x in meta_batch.loc[:, ~meta_batch.columns.isin(['id'])].values.tolist()]
    # create Cohere vectors
    embeds = embeddings.embed_documents([text for text in meta_batch])
    # create unique IDs
    ids = [str(x) for x in range(i, i_end)]

    upserts = []
    # loop through the data and create dictionaries for uploading documents to pinecone index
    for _id, dense, meta in zip(ids, embeds, meta_dict):
        upserts.append({
            'id': _id,
            'values': dense,
            'metadata': meta
        })
    # upload the documents to the new hybrid index
    index.upsert(upserts)

# show index description after uploading the documents
index.describe_index_stats()

  0%|          | 0/174 [00:00<?, ?it/s]

{'dimension': 1024,
 'index_fullness': 0.2,
 'namespaces': {'': {'vector_count': 17396}},
 'total_vector_count': 17396}

## Creating a Vector Store and Querying

In [60]:
from langchain.vectorstores import Pinecone

text_field = "description"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = Pinecone(
    index, embeddings.embed_query, text_field
)

In [58]:
query = "I am looking for room with 2 beds and private bathroom"

vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

[Document(page_content='Hi, everybody. My house is located in Xiafei Road and is an old-fashioned apartment. I share a room with a double bed and some living items that can be used by singles or couples. The living room, bathroom and toilet are shared with me.', metadata={'accommodates': '2', 'amenities': 'TV, Essentials, Kitchen, Private living room, Lock on bedroom door, Fire extinguisher, Elevator, Pets allowed, Smoking allowed, Dryer, Hangers, Shampoo, Hair dryer', 'bathrooms': '1 shared bath', 'bedrooms': '1', 'beds': '1', 'host_about': "Hi, everybody. I'm very glad to meet you. I like swimming and fitness and I hope to be friends with everyone.", 'host_is_superhost': 'No', 'host_name': 'Woranuch', 'host_since': datetime.date(2018, 4, 21), 'id': '24607801', 'maximum_nights': '1125', 'minimum_nights': '1', 'name': 'Rooms Double Beds Apartment', 'neighborhood_overview': '', 'neighbourhood': 'Bang Phlat', 'picture_url': 'https://a0.muscache.com/pictures/5844169c-de81-454e-83ba-2a5ed6